In [1]:
import os, json, gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import string
import nltk

from nltk.corpus import wordnet, stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

In [2]:
df = pd.read_csv('Book_1_50.csv')
# Check the population size (number of rows) in your DataFrame
print("Population size:", len(df))


Population size: 43148


In [3]:
# drop any rows w/ missing values
df = df.dropna(subset=["reviewText"])
print("Population size:", len(df))
#discover the actual counts
print(df.overall.value_counts())

print("Population size:", len(df))

# set sample size to labels w/ minimum count
sample_size = 612
# Collect samples for each class in a list
samples_list = [df[df.overall == label].sample(n=sample_size, random_state=1) for label in df.overall.unique()]

# Concatenate all the samples into one DataFrame
df_equal_overall = pd.concat(samples_list, ignore_index=True)

Population size: 43131
5.0    34452
4.0     5514
3.0     1861
2.0      692
1.0      612
Name: overall, dtype: int64
Population size: 43131


In [4]:
stopwords_list = stopwords.words('english')

def ReviewProcessing(df):
  # remove non alphanumeric
  df['review_cleaned'] = df.reviewText.str.replace('[^a-zA-Z0-9 ]', '')
  # lowercase
  df.review_cleaned = df.review_cleaned.str.lower()
  # split into list
  df.review_cleaned = df.review_cleaned.str.split(' ')
  # remove stopwords
  df.review_cleaned = df.review_cleaned.apply(lambda x: [item for item in x if item not in stopwords_list])
  return df

In [5]:
def get_wordnet_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

  return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = nltk.stem.WordNetLemmatizer()
def get_lemmatize(sent):
  return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])


In [6]:
clean_data = ReviewProcessing(df_equal_overall)
clean_data.review_cleaned = clean_data.review_cleaned.apply(' '.join)
clean_data['review_cleaned_lemmatized'] = clean_data.review_cleaned.apply(get_lemmatize)

In [7]:
from sklearn.naive_bayes import MultinomialNB

nb = Pipeline([('vectorize', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [8]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier()),
               ])

In [9]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=500)),
               ])

In [10]:
x = clean_data['review_cleaned_lemmatized']
y = clean_data['overall']
X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2, stratify=y, random_state = 44)


In [11]:
# Naive Bayes
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print(accuracy_score(y_test, y_pred_nb))
print(confusion_matrix(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# SGD Classifier
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
print(accuracy_score(y_test, y_pred_sgd))
print(confusion_matrix(y_test, y_pred_sgd))
print(classification_report(y_test, y_pred_sgd))

# Logistic Regression
logreg.fit(X_train, y_train)
y_pred_log = logreg.predict(X_test)
print(accuracy_score(y_test, y_pred_log))
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

0.4411764705882353
[[60 42  8  9  3]
 [20 64 14 11 13]
 [10 47 38 20  7]
 [ 6 29 12 41 35]
 [ 4 18 11 23 67]]
              precision    recall  f1-score   support

         1.0       0.60      0.49      0.54       122
         2.0       0.32      0.52      0.40       122
         3.0       0.46      0.31      0.37       122
         4.0       0.39      0.33      0.36       123
         5.0       0.54      0.54      0.54       123

    accuracy                           0.44       612
   macro avg       0.46      0.44      0.44       612
weighted avg       0.46      0.44      0.44       612

0.43300653594771243
[[66 20 13 12 11]
 [29 43 18 19 13]
 [11 27 39 29 16]
 [ 6 10 23 45 39]
 [ 6  4 16 25 72]]
              precision    recall  f1-score   support

         1.0       0.56      0.54      0.55       122
         2.0       0.41      0.35      0.38       122
         3.0       0.36      0.32      0.34       122
         4.0       0.35      0.37      0.36       123
         5.0       

In [12]:

from sklearn.model_selection import GridSearchCV

grid=[{'clf__solver': ['lbfgs', 'sag', 'saga'],
       'clf__C': [0.01, 0.1, 1]}]
lr = GridSearchCV(logreg, param_grid = grid, cv = 5, scoring='accuracy', verbose = 1, n_jobs = -1)
best_model = lr.fit(X_train, y_train)

print(best_model.best_estimator_)
print(best_model.best_score_)

y_pred_grid = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid))
print(accuracy_score(y_test, y_pred_grid))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  45 | elapsed:    5.8s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    9.0s finished


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
    